# Manifold GP Supervised Learning via Precision Matrix on 1D Manifold

## Preamble

This notebook provides an example of how to perform Gaussian Process Regression on a 1D manifold. In this example we consider a supervised learning scenario, namely the number of labeled data points is equivalent to the number of the sampled points from the underlying manifold.

In [1]:
import torch
import gpytorch
import numpy as np
import os
import scipy.spatial as ss

from manifold_gp.kernels.riemann_matern_kernel import RiemannMaternKernel
from manifold_gp.models.riemann_gp import RiemannGP
from gpytorch.priors import NormalPrior, GammaPrior

## Dataset Preprocessing

### Load & Settings

In [2]:
dataset = 'protein' # ['protein','elevators', 'ctslice']
cut = 10000

if dataset == 'protein':
    data = np.loadtxt('datasets/protein.csv', delimiter=",")[:cut]
    sampled_x, sampled_y = data[:, 1:], data[:, 0]
elif dataset == 'elevators':
    data = np.array(loadmat('datasets/elevators.mat')['data'])
    sampled_x, sampled_y = data[:, :-1], data[:, -1]
elif dataset == 'ctslice':
    data = np.loadtxt('datasets/ctslice.csv', delimiter=",")[:cut]
    sampled_x, sampled_y = data[:, :-1], data[:, -1]
    
# remove coincident points
sampled_x, id_unique = np.unique(sampled_x, axis=0, return_index=True)
sampled_y = sampled_y[id_unique]

# cut between 0.01 and 0.99 quantile of distances
kd_tree = ss.KDTree(sampled_x)
v = kd_tree.query(sampled_x, k=2)[0][:, 1]
idx = np.argsort(v)
percentile_start = int(np.round(idx.shape[0]*0.10))
percentile_end = int(np.round(idx.shape[0]*0.90))
sampled_x = sampled_x[idx[percentile_start:percentile_end], :]
sampled_y = sampled_y[idx[percentile_start:percentile_end]]
m = sampled_x.shape[0]

normalize_features = True
normalize_labels = True

### Trainset & Testset

In [3]:
split = int(0.2 * m)

train_x, train_y = sampled_x[:split], sampled_y[:split]
test_x, test_y = sampled_x[split:], sampled_y[split:]

train_x = torch.from_numpy(train_x).float()
train_y = torch.from_numpy(train_y).float()
test_x = torch.from_numpy(test_x).float()
test_y = torch.from_numpy(test_y).float()

if normalize_features:
    mu_x, std_x = train_x.mean(dim=-2, keepdim=True), train_x.std(dim=-2, keepdim=True) + 1e-6
    train_x.sub_(mu_x).div_(std_x)
    test_x.sub_(mu_x).div_(std_x)
    
if normalize_labels:
    mu_y, std_y = train_y.mean(), train_y.std()
    train_y.sub_(mu_y).div_(std_y)
    test_y.sub_(mu_y).div_(std_y)

### Hyperparameters Priors

In [4]:
import scipy.spatial as ss
neighbors = 10
kd_tree = ss.KDTree(train_x)
v = np.sort(kd_tree.query(train_x, k=neighbors+1)[0][:, 1:].ravel())
percentile_99 = int(np.round(v.shape[0]*0.99))
gamma_rate = 100.0/np.std(v)
gamma_concentration = gamma_rate * v[percentile_99] + 1

### Move Data to Device

In [5]:
train_x, train_y = train_x.contiguous(), train_y.contiguous()
test_x, test_y = test_x.contiguous(), test_y.contiguous()

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
train_x, train_y = train_x.to(device), train_y.to(device)
test_x, test_y = test_x.to(device), test_y.to(device)

## Model

In [6]:
%%capture
likelihood = gpytorch.likelihoods.GaussianLikelihood(
    noise_constraint=gpytorch.constraints.GreaterThan(1e-8),
    noise_prior=None  # NormalPrior(torch.tensor([0.0]).to(device),  torch.tensor([1/9]).sqrt().to(device))
)

kernel = gpytorch.kernels.ScaleKernel(
    RiemannMaternKernel(
        nu=3,
        nodes=train_x,
        neighbors=50,
        operator="randomwalk",
        modes=200,
        ball_scale=3.0,
        support_kernel=gpytorch.kernels.RBFKernel(),
        epsilon_prior=GammaPrior(gamma_concentration, gamma_rate),
        lengthscale_prior=None  # InverseGammaPrior(igamma_concentration, igamma_rate)
    ),
    outputscale_prior=None  # NormalPrior(torch.tensor([1.0]).to(device),  torch.tensor([1/9]).sqrt().to(device))
)

model = RiemannGP(train_x, train_y, likelihood, kernel).to(device)

## Train

In [7]:
%%capture
hypers = {
    'likelihood.noise_covar.noise': 1e-2,
    'covar_module.base_kernel.epsilon': 0.5,
    'covar_module.base_kernel.lengthscale': 0.5,
    'covar_module.outputscale': 1.0,
    'covar_module.base_kernel.support_kernel.lengthscale': 1.0,
}
model.initialize(**hypers)

In [8]:
model.manifold_informed_train(lr=1e-2, iter=100, verbose=True)

Iter: 0, LR: 0.010, Loss: 6321.388, NoiseVar: 0.010, SignalVar: 38601.328, Lengthscale: 0.500, Epsilon: 0.500
Iter: 1, LR: 0.010, Loss: 6011.245, NoiseVar: 0.010, SignalVar: 38601.340, Lengthscale: 0.504, Epsilon: 0.496
Iter: 2, LR: 0.010, Loss: 5725.534, NoiseVar: 0.010, SignalVar: 38601.352, Lengthscale: 0.508, Epsilon: 0.492
Iter: 3, LR: 0.010, Loss: 5451.588, NoiseVar: 0.010, SignalVar: 38601.363, Lengthscale: 0.512, Epsilon: 0.488
Iter: 4, LR: 0.010, Loss: 5197.344, NoiseVar: 0.010, SignalVar: 38601.375, Lengthscale: 0.516, Epsilon: 0.485
Iter: 5, LR: 0.010, Loss: 4943.492, NoiseVar: 0.011, SignalVar: 38601.387, Lengthscale: 0.520, Epsilon: 0.481
Iter: 6, LR: 0.010, Loss: 4727.778, NoiseVar: 0.011, SignalVar: 38601.398, Lengthscale: 0.524, Epsilon: 0.477
Iter: 7, LR: 0.010, Loss: 4494.203, NoiseVar: 0.011, SignalVar: 38601.406, Lengthscale: 0.528, Epsilon: 0.473
Iter: 8, LR: 0.010, Loss: 4299.874, NoiseVar: 0.011, SignalVar: 38601.414, Lengthscale: 0.532, Epsilon: 0.470
Iter: 9, L

Iter: 75, LR: 0.010, Loss: 1442.481, NoiseVar: 0.014, SignalVar: 38601.785, Lengthscale: 0.690, Epsilon: 0.371
Iter: 76, LR: 0.010, Loss: 1449.238, NoiseVar: 0.014, SignalVar: 38601.789, Lengthscale: 0.691, Epsilon: 0.371
Iter: 77, LR: 0.010, Loss: 1441.552, NoiseVar: 0.014, SignalVar: 38601.793, Lengthscale: 0.692, Epsilon: 0.370
Iter: 78, LR: 0.010, Loss: 1449.138, NoiseVar: 0.014, SignalVar: 38601.797, Lengthscale: 0.694, Epsilon: 0.370
Iter: 79, LR: 0.010, Loss: 1426.109, NoiseVar: 0.014, SignalVar: 38601.801, Lengthscale: 0.695, Epsilon: 0.370
Iter: 80, LR: 0.010, Loss: 1438.204, NoiseVar: 0.014, SignalVar: 38601.805, Lengthscale: 0.696, Epsilon: 0.370
Iter: 81, LR: 0.010, Loss: 1428.155, NoiseVar: 0.014, SignalVar: 38601.809, Lengthscale: 0.697, Epsilon: 0.370
Iter: 82, LR: 0.010, Loss: 1420.219, NoiseVar: 0.014, SignalVar: 38601.812, Lengthscale: 0.698, Epsilon: 0.370
Iter: 83, LR: 0.010, Loss: 1416.422, NoiseVar: 0.014, SignalVar: 38601.816, Lengthscale: 0.699, Epsilon: 0.371
I

In [9]:
model.manifold_informed_train(lr=1e-1, iter=100, verbose=True)

Iter: 0, LR: 0.100, Loss: 1996.456, NoiseVar: 0.015, SignalVar: 19353.910, Lengthscale: 0.716, Epsilon: 0.374
Iter: 1, LR: 0.100, Loss: 1636.002, NoiseVar: 0.016, SignalVar: 19354.010, Lengthscale: 0.768, Epsilon: 0.344
Iter: 2, LR: 0.100, Loss: 1484.708, NoiseVar: 0.018, SignalVar: 19354.105, Lengthscale: 0.820, Epsilon: 0.331
Iter: 3, LR: 0.100, Loss: 1400.689, NoiseVar: 0.020, SignalVar: 19354.195, Lengthscale: 0.869, Epsilon: 0.338
Iter: 4, LR: 0.100, Loss: 1299.514, NoiseVar: 0.021, SignalVar: 19354.279, Lengthscale: 0.917, Epsilon: 0.354
Iter: 5, LR: 0.100, Loss: 1264.104, NoiseVar: 0.023, SignalVar: 19354.355, Lengthscale: 0.960, Epsilon: 0.375
Iter: 6, LR: 0.100, Loss: 1212.696, NoiseVar: 0.024, SignalVar: 19354.424, Lengthscale: 1.000, Epsilon: 0.400
Iter: 7, LR: 0.100, Loss: 1196.493, NoiseVar: 0.026, SignalVar: 19354.482, Lengthscale: 1.036, Epsilon: 0.427
Iter: 8, LR: 0.100, Loss: 1175.473, NoiseVar: 0.027, SignalVar: 19354.531, Lengthscale: 1.066, Epsilon: 0.457
Iter: 9, L

Iter: 80, LR: 0.100, Loss: 782.019, NoiseVar: 0.037, SignalVar: 19353.953, Lengthscale: 0.939, Epsilon: 2.152
Iter: 81, LR: 0.100, Loss: 783.083, NoiseVar: 0.037, SignalVar: 19353.949, Lengthscale: 0.939, Epsilon: 2.150
Iter: 82, LR: 0.100, Loss: 783.512, NoiseVar: 0.037, SignalVar: 19353.947, Lengthscale: 0.939, Epsilon: 2.147
Iter: 83, LR: 0.100, Loss: 782.811, NoiseVar: 0.036, SignalVar: 19353.945, Lengthscale: 0.939, Epsilon: 2.144
Iter: 84, LR: 0.100, Loss: 781.651, NoiseVar: 0.036, SignalVar: 19353.943, Lengthscale: 0.940, Epsilon: 2.141
Iter: 85, LR: 0.100, Loss: 782.074, NoiseVar: 0.036, SignalVar: 19353.941, Lengthscale: 0.941, Epsilon: 2.137
Iter: 86, LR: 0.100, Loss: 783.559, NoiseVar: 0.036, SignalVar: 19353.939, Lengthscale: 0.943, Epsilon: 2.134
Iter: 87, LR: 0.100, Loss: 781.693, NoiseVar: 0.036, SignalVar: 19353.938, Lengthscale: 0.943, Epsilon: 2.131
Iter: 88, LR: 0.100, Loss: 783.312, NoiseVar: 0.036, SignalVar: 19353.936, Lengthscale: 0.944, Epsilon: 2.127
Iter: 89, 

## Evaluation

In [10]:
%%capture
likelihood.eval()
model.eval()

## Metrics

In [11]:
with torch.no_grad(), gpytorch.settings.fast_pred_var(), gpytorch.settings.cg_tolerance(10000):
    preds_test = likelihood(model(test_x))
        
    error = test_y - preds_test.mean
    covar = preds_test.lazy_covariance_matrix.evaluate_kernel()
    inv_quad, logdet = covar.inv_quad_logdet(inv_quad_rhs=error.unsqueeze(-1), logdet=True)
    
    rmse = (error.square().sum()/test_y.shape[0]).sqrt()
    nll = 0.5 * sum([inv_quad, logdet, error.size(-1)* np.log(2 * np.pi)])/test_y.shape[0]
    
print("RMSE: ", rmse)
print("NLL: ", nll)

RMSE:  tensor(1.0302, device='cuda:0')
NLL:  tensor(10.4026, device='cuda:0')
